## Walk through of a sample PCL code:

### Lesson 1 S3 User and Bucket configuration Example

[object.py](../../object.py)

#### The Header Comment Section

This section usually includes details such as 
- the file name, 
- author, 
- date, 
- purpose of the program, and any other relevant information. 

It’s a good practice to include such comments for better readability and maintainability of the code 

In [ ]:
#! /usr/local/bin/python3

"""
ONTAP 9.13.1 REST API Python Client Library Scripts
Author: Vish Hulikal
This script performs the following:
        - Create an SVM (or VServer)
        - Create an S3 user
        - Create an S3 bucket
Minimum size of the bucket to be created: 102005473280B
usage: python3.11 object.py [-h] -c CLUSTER -vs VSERVER_NAME -a AGGR_NAME -s S3_NAME -b BUCKET_NAME
               -un S3_USER_NAME -n SIZE [-u API_USER] [-p API_PASS]
These arguments required: -c/--cluster, -vs/--vserver_name, -a/--aggr_name, -s/--s3_name, -b/--bucket_name
               -un/--s3_user_name, -n/--size, -u/--admin, -p/--password
Usage: python3.11 object.py -c cluster1 -vs VServer1 -a aggr1 -s S3_Server -b s3bucket -un S3_User -n 102005473280 -u admin -p Netapp1!
"""


#### The Input Section:

Since we are using a Jupyter Notebook to run the code and not running the code from the command line, we need this section to simulate entering the arguments from the command line.

When you run the notebook, for example by clicking on the `Run All` button, a dialog prompt will appear at the top of the window. You will then need to enter the command line arguments as described in the usage section above.

If you do not provide the user, `admin`is used. If you do not provide the password, you will be prompted for the password.

Suggested command arguments:

In [ ]:
import sys

# Prompt the user to enter command line arguments
args = input("Please enter command line arguments: ")

# Split the entered string into a list of arguments
args = args.split()

# Assign the list of arguments to sys.argv
sys.argv = ['ipykernel_launcher.py'] + args

#### Import Section

Here we will import the following modules:
  - [`argparse`](https://pypi.org/project/argparse/) : This is a popular python module. The argparse module makes it easy to write user friendly command line interfaces. 
  The program defines what arguments it requires, and argparse will figure out how to parse those out of sys.argv. The argparse module also automatically generates help and usage messages and issues errors when users give the program invalid arguments.
  - [`getpass`](https://docs.python.org/3/library/getpass.html) : Used to Prompt the user for a password without echoing
  - [`logging`](https://pypi.org/project/logging/) : This module is intended to provide a standard error logging mechanism in Python as per PEP 282.
  - [`netapp_ontap.config`](https://library.netapp.com/ecmdocs/ECMLP3319064/html/config.html) : This module contains the global configuration options and related functions for the library.
  - [`netapp_ontap.host_connection`](https://library.netapp.com/ecmdocs/ECMLP3319064/html/host_connection.html) : This module defines a host connection object which is used to communicate with the API host
  - [`netapp_ontap.error`](https://library.netapp.com/ecmdocs/ECMLP3319064/html/error.html) : This module defines the custom exception type. All exceptions raised by the library descend from this type
  - [`netapp_ontap.resources.svm`](https://library.netapp.com/ecmdocs/ECMLP3319064/html/resources/svm.html) : Managing SVMs
  - [`netapp_ontap.resources.s3_bucket`](https://library.netapp.com/ecmdocs/ECMLP2885777/html/resources/s3_bucket.html) : S3 Bucket  
  - [`netapp_ontap.resources.s3_user`](https://library.netapp.com/ecmdocs/ECMLP2885777/html/resources/s3_user.html) : S3 User


In [ ]:

import argparse
from getpass import getpass
import logging

from netapp_ontap import config, HostConnection, NetAppRestError
from netapp_ontap.resources import Aggregate, Svm, S3BucketSvm, S3User


#### Function Definitions

These functions are defined:
  
  - `create_svm`
  - `make_s3_user`  
  - `make_bucket`  

  

In [ ]:
def create_svm(vserver_name: str, aggr_name: str, S3_name: str) -> None:
    """Create an SVM on the specified aggregate and configures an S3 server.
       access_key and secret_key - are returned from the Post call."""

    svm = Svm.from_dict({
      'name': vserver_name,
      'aggregates': [{'name': aggr_name}],
      's3.name': S3_name,
      'enabled': "true",
    })

    try:
        svm.post()
        print("SVM %s created successfully" % svm.name)
    except NetAppRestError as err:
        print("Error: SVM was not created: %s" % err)
    return


In [ ]:

def make_s3_user(vserver_name: str, user_name: str) -> None:
    """Creates a new S3 user configuration"""

    vserver = Svm.find(name=vserver_name)
    resource = S3User(vserver.uuid)
    resource.name = user_name

    try:
        resource.post()
        print("S3 User %s created successfully" % resource.name)
    except NetAppRestError as err:
        print("Error: S3 User was not created" % err)
    return


In [ ]:

def make_bucket(vserver_name: str, aggr_name: str, bucket_name: str, bucket_size: int) -> None:
    """Make an S3 Bucket"""

    vserver = Svm.find(name=vserver_name)
    aggregate = Aggregate.find(name=aggr_name)
    resource = S3BucketSvm(vserver.uuid)
    resource.name = bucket_name
    resource.comment = "S3 Bucket"
    resource.aggregates = [
        {'name': aggr_name, 'uuid': aggregate.uuid}
    ]
    resource.constituents_per_aggregate = 4
    resource.size = bucket_size

    try:
        resource.post()
        print("S3 Bucket %s created successfully" % resource.name)
    except NetAppRestError as err:
        print("Error: S3 Bucket was not created: %s" % err)
    return


#### Arguments Parser

We define which arguments need to be passed to the script and argparse does the rest...

In [ ]:
def parse_args() -> argparse.Namespace:
    """Parse the command line arguments from the user"""

    parser = argparse.ArgumentParser(
        description="This script will create an SVM, an S3 User and an S3 bucket."
    )
    parser.add_argument(
        "-c", "--cluster", required=True, help="API server IP:port details"
    )
    parser.add_argument(
        "-vs", "--vserver_name", required=True, help="SVM name"
    )
    parser.add_argument(
        "-a", "--aggr_name", required=True, help="Aggregate name"
    )
    parser.add_argument(
        "-s", "--s3_name", required=True, help="S3 Server name"
    )
    parser.add_argument(
        "-b", "--bucket_name", required=True, help="Bucket name"
    )
    parser.add_argument(
        "-un", "--user_name", required=True, help="S3 User name"
    )
    parser.add_argument(
        "-n", "--size", required=True, help="Size of the bucket in bytes"
    )
    parser.add_argument("-u", "--api_user", default="admin", help="API Username")
    parser.add_argument("-p", "--api_pass", help="API Password")
    parsed_args = parser.parse_args()

    # collect the password without echo if not already provided
    if not parsed_args.api_pass:
        parsed_args.api_pass = getpass()

    return parsed_args


#### The Main Section

In [ ]:
if __name__ == "__main__":
    logging.basicConfig(
        level=logging.INFO,
        format="[%(asctime)s] [%(levelname)5s] [%(module)s:%(lineno)s] %(message)s",
    )
    args = parse_args()
    config.CONNECTION = HostConnection(
        args.cluster, username=args.api_user, password=args.api_pass, verify=False,
    )

    # Create a VServer, an S3 user and a Bucket
    create_svm(args.vserver_name, args.aggr_name, args.s3_name)
    make_s3_user(args.vserver_name, args.user_name)
    make_bucket(args.vserver_name, args.aggr_name, args.bucket_name, args.size)


---

---

[Back to Module 5 Demo - Notebook:](./Module5_demo.ipynb)